In [157]:
import feedparser
import hashlib
from threading import Timer
import time
from pandas import DataFrame
import datetime
import pandas as pd
import os.path

dump_file = "news.csv"

def open_dump(path):
    if os.path.isfile(path):
        return pd.read_csv(path)

def scrap(hashes):
    ts = int(time.time())
    dt = pd.to_datetime(datetime.datetime.now())
    data_temp = []
    feed_raw = feedparser.parse('https://fivethirtyeight.com/features/feed/')
    feed_entries = feed_raw['entries']

    for f in feed_entries:
        title = f['title'].encode('utf-8')
        hashed_title = hashlib.md5(title).hexdigest()

        if not hashed_title in hashes:
            entry = {'dt': dt, 'ts' : ts, 'title' :  title, 'hash': hashed_title} 
            data_temp.append(entry)

    df = DataFrame.from_records(data_temp, columns = ['dt', 'ts', 'title', 'hash']) 
    #df.columns
    return df

def run_dump():
    data = open_dump(dump_file)
    if data is None or data.empty:
        print "No CSV dump"
        # first start without csv
        res = scrap([])
        res.to_csv(path_or_buf=dump_file, columns = ['dt', 'ts', 'title', 'hash'])
    else:
        hashes = data['hash'].tolist()
        res = scrap(hashes)
        data = data.append(res, ignore_index=True)
        data.to_csv(path_or_buf=dump_file, columns = ['dt', 'ts', 'title', 'hash'])
    Timer(10, run_dump).start()

def run():
    run_dump()        
    

In [158]:
run_dump()

In [159]:
def read_dump():
    return open_dump(dump_file)

In [163]:
df = read_dump()
df = df.set_index(['dt'])
result = df.loc['2017-01-23':'2017-01-24']
result



,Unnamed: 0,ts,title,hash
dt,,,,
2017-01-23 14:34:03.522055,0,1.485175e+09,The Invisible Undecided Voter,7cbde96fd64695f9a2b7f93dcf4e469b
2017-01-23 14:34:03.522055,1,1.485175e+09,It Wasn’t Clinton’s Election To Lose,0d4263ffb58b286303cbbc5ecc7a9735
2017-01-23 14:34:03.522055,2,1.485175e+09,The Electoral College Blind Spot,c374e130130c442c8f403f687b5e7978
2017-01-23 14:34:03.522055,3,1.485175e+09,Trump’s Executive Order On Obamacare Means Eve...,b219db19176cacf6f982500851dd4974
2017-01-23 14:34:03.522055,4,1.485175e+09,How Unconscious Sexism Could Help Explain Trum...,de3c7871e66c42bb27fb5007bc9e269d
2017-01-23 14:34:03.522055,5,1.485175e+09,How To Win With Carmelo Anthony,6c8cb5129f96b350f7cf2b6000fece3c
2017-01-23 14:34:03.522055,6,1.485175e+09,The Weaknesses Of The Four Remaining NFL Playo...,fa8e8e4687ade010d5b37b3dad9037ff
2017-01-23 14:34:03.522055,7,1.485175e+09,Women Still Don’t Like Trump,3f9eb5ea59c026d61d94031da07111ab
2017-01-23 14:34:03.522055,8,1.485175e+09,NFL Conference Championship Previews: Are Matt...,86fefbeb0f457e5ac42ed7a014051354
